In [1]:
import pandas as pd
import numpy as np
import time
from pathlib import Path

print("COMPLETE: Setup complete")

COMPLETE: Setup complete


In [2]:
# Specify latest season data needed for and how many season to get in total e.g. 21 and 5 would get 2021 and the 4 previous seasons data (5 in total)
latest_season = 22
number_of_seasons = 2

# Select which new data items to obtain
get_new_games = True
get_new_pbp = False
get_new_rosters = True
get_new_shotcharts = True
get_new_player_gamelogs = True

print("COMPLETE: Notebook configured")

COMPLETE: Notebook configured


In [3]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
import json

In [30]:
#### Get player game logs
### Retrieves details of each players statistics in each game
from nba_api.stats.endpoints import playergamelogs

master_player_game_logs = []
if get_new_player_gamelogs:
    # Iterate through the seasons and save each season to a csv
    for n in range(0, number_of_seasons):
        # construct the season name for the request
        season_name = f"20{latest_season-(n+1)}-{latest_season-n}"
        # request the gamelogs for the season
        playergamefinder = playergamelogs.PlayerGameLogs(season_nullable=season_name, league_id_nullable='00')
        player_game_df = playergamefinder.get_data_frames()
        # save the returned results to csv
        player_game_df[0].to_csv(f"Data/player_gamelogs_20{latest_season-n}.csv")
        
#         ## do with json instead
#         player_game_json = json.loads(playergamefinder.get_normalized_json())
#         master_player_game_logs.extend(player_game_json['PlayerGameLogs'])
        
#         # Serializing json
#         json_object = json.dumps(master_player_game_logs[0], indent=4)


    
        print(f"Player gamelogs obtained for 20{latest_season-n}")
    
    # Writing to sample.json
#     with open("master_player_game_logs.json", "w") as outfile:
#         outfile.write(json_object)
#     print("COMPLETE: Player gamelogs obtained")
else:
    print("Player gamelogs not requested (as per configuration")

Player gamelogs obtained for 2022
Player gamelogs obtained for 2021


In [32]:
#### Get the rosters
### Retrieves the player rosters for teams
### Does not need to be run often

from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster

if get_new_rosters:
    # Get the team ids
    nba_teams = teams.get_teams()
    nba_team_ids = [team['id'] for team in nba_teams]
    # Iterate through the required seasons and teams to get the rosters, save each season to a csv
    for n in range(0, number_of_seasons):
        season_name = f"20{latest_season-(n+1)}-{latest_season-n}"
        rosters = []
        for team in nba_team_ids:
            # throttles requests to prevent api from blocking them
            time.sleep(.600)
            roster = commonteamroster.CommonTeamRoster(team_id=team, season=season_name, timeout=1000).get_data_frames()[0]
            rosters.append(roster)
        # concatenate all the returned entries and save to csv
        season_rosters = pd.concat(rosters, axis=0, ignore_index=True)
        season_rosters.to_csv(f"Data/rosters_20{latest_season-n}.csv")
        print(f"All rosters loaded for 20{latest_season-n}")
    print("COMPLETE: Rosters loaded")
else:
    print("Rosters not requested (as per configuration)")

All rosters loaded for 2022
All rosters loaded for 2021
COMPLETE: Rosters loaded


In [117]:
rosters_df

,Unnamed: 0,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,0,1610612737,2021,0,Delon Wright,Delon,delon-wright,0.0,G,6-5,185,"APR 26, 1992",30.0,6,Utah,1626153,NaN
1,1,1610612737,2021,0,Jalen Johnson,Jalen,jalen-johnson,1.0,F,6-8,219,"DEC 18, 2001",20.0,R,Duke,1630552,#20 Pick in 2021 Draft
2,2,1610612737,2021,0,Sharife Cooper,Sharife,sharife-cooper,2.0,G,6-1,176,"JUN 11, 2001",21.0,R,Auburn,1630536,NaN
3,3,1610612737,2021,0,Kevin Huerter,Kevin,kevin-huerter,3.0,G-F,6-7,198,"AUG 27, 1998",23.0,3,Maryland,1628989,NaN
4,4,1610612737,2021,0,Skylar Mays,Skylar,skylar-mays,4.0,G,6-4,205,"SEP 05, 1997",24.0,1,Louisiana State,1630219,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,501,1610612766,2021,0,JT Thor,JT,jt-thor,21.0,F,6-9,203,"AUG 26, 2002",19.0,R,Auburn,1630550,Draft Rights Traded from DET on 08/06/21
502,502,1610612766,2021,0,Kai Jones,Kai,kai-jones,23.0,C-F,6-10,221,"JAN 19, 2001",21.0,R,Texas,1630539,Draft Rights Traded from NYK on 07/30/21
503,503,1610612766,2021,0,Mason Plumlee,Mason,mason-plumlee,24.0,F-C,6-11,254,"MAR 05, 1990",32.0,8,Duke,203486,Traded from DET on 08/06/21
504,504,1610612766,2021,0,P.J. Washington,P.J.,pj-washington,25.0,F,6-7,230,"AUG 23, 1998",23.0,2,Kentucky,1629023,#12 Pick in 2019 Draft


In [118]:
#### Get the shot charts
### Retrieves shot chart based on team and player (mandatory inputs)
from nba_api.stats.endpoints import shotchartdetail

if get_new_shotcharts:
    for n in range(0, number_of_seasons):
        # Load the csv containing the rosters
        rosters_file = Path(f"Data/rosters_20{latest_season-n}.csv")
        if rosters_file.is_file():
            rosters_df = pd.read_csv(rosters_file, index_col=None, header=0, low_memory=False)
            shotcharts = []
            # Iterate through the players and teams to get shot charts 
            # *may end up with duplicate shot charts where a player is at the same team more than one season
            for row in rosters_df.itertuples():
                try:
                    player_id = row.PLAYER_ID
                    team_id = row.TeamID
                    # throttles requests to prevent api from blocking them
                    time.sleep(.600)
                    # requests shotchartdetail for player and team with context field goals attempted (FGA)
                    sc_df = shotchartdetail.ShotChartDetail(player_id=player_id, team_id=team_id, context_measure_simple='FGA').get_data_frames()[0]
                    shotcharts.append(sc_df)
                except: 
                    pass
            # concatenate the results together and save to csv
            season_shotchart = pd.concat(shotcharts, axis=0, ignore_index=True)
            season_shotchart.to_csv(f"Data/shotchart_20{latest_season-n}.csv")

In [77]:
df_ = shotchartdetail.ShotChartDetail(player_id=player_id, team_id=team_id, context_measure_simple='FGA')





In [105]:
dir(df_.shot_chart_detail)
# pd.DataFrame()
# dir()
dir(df_.shot_chart_detail)#.get_data_frame)
df_.shot_chart_detail.get_dict()#shot_chart_detail.data

{'headers': ['GRID_TYPE',
  'GAME_ID',
  'GAME_EVENT_ID',
  'PLAYER_ID',
  'PLAYER_NAME',
  'TEAM_ID',
  'TEAM_NAME',
  'PERIOD',
  'MINUTES_REMAINING',
  'SECONDS_REMAINING',
  'EVENT_TYPE',
  'ACTION_TYPE',
  'SHOT_TYPE',
  'SHOT_ZONE_BASIC',
  'SHOT_ZONE_AREA',
  'SHOT_ZONE_RANGE',
  'SHOT_DISTANCE',
  'LOC_X',
  'LOC_Y',
  'SHOT_ATTEMPTED_FLAG',
  'SHOT_MADE_FLAG',
  'GAME_DATE',
  'HTM',
  'VTM'],
 'data': []}

In [114]:
dir(df_)
# dir(df_.league_averages)
# df_.league_averages.get_dict()
df_.player_stats

In [119]:
season_shotchart

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022200005,10,1629027,Trae Young,1610612737,Atlanta Hawks,1,11,14,...,Center(C),24+ ft.,36,1,363,1,0,20221019,ATL,HOU
1,Shot Chart Detail,0022200005,43,1629027,Trae Young,1610612737,Atlanta Hawks,1,8,17,...,Center(C),8-16 ft.,10,39,95,1,0,20221019,ATL,HOU
2,Shot Chart Detail,0022200005,61,1629027,Trae Young,1610612737,Atlanta Hawks,1,6,45,...,Left Side Center(LC),24+ ft.,27,-89,263,1,0,20221019,ATL,HOU
3,Shot Chart Detail,0022200005,69,1629027,Trae Young,1610612737,Atlanta Hawks,1,6,26,...,Center(C),8-16 ft.,10,31,102,1,0,20221019,ATL,HOU
4,Shot Chart Detail,0022200005,81,1629027,Trae Young,1610612737,Atlanta Hawks,1,5,33,...,Center(C),Less Than 8 ft.,2,-2,20,1,1,20221019,ATL,HOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55610,Shot Chart Detail,0022200583,513,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,11,3,...,Center(C),Less Than 8 ft.,6,43,48,1,1,20230106,MIL,CHA
55611,Shot Chart Detail,0022200583,563,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,7,40,...,Right Side Center(RC),24+ ft.,26,123,232,1,1,20230106,MIL,CHA
55612,Shot Chart Detail,0022200583,575,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,6,26,...,Left Side Center(LC),24+ ft.,25,-154,205,1,0,20230106,MIL,CHA
55613,Shot Chart Detail,0022200583,596,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,4,33,...,Left Side(L),24+ ft.,23,-230,1,1,1,20230106,MIL,CHA
